# 전처리 파일

MeCab으로 전처리를 진행하는 코드



In [ ]:
import pandas as pd     # csv 파일 편집
import MeCab            # 전처리
import re               # 정규 표현식 (전처리)


In [ ]:
csv_test = pd.read_csv('./SampleData/yeonhap.csv')
csv_test.shape

In [ ]:
# 데이터 확인 데이터 확인
csv_test.head

In [ ]:
#csv_test.columns

In [ ]:
#csv_test["Contents"]

In [ ]:
# 원문 확인
#csv_test["Contents"][0]

## 동작 부분

- 한 줄 씩 CSV 파일로 저장하며 진행하려면 .to_csv(mode='a')를 해줘야 한다. append 모드를 사용하겠다는 뜻

- 자꾸 새로운 column 인 index 가 생기는 문제
    index=False 를 옵션으로 추가한다.

- 한 줄 추가 할때 마다 column 들이 새로 만들어짐
    - header=False 를 옵션으로 추가한다.
        - 단, column 정보가 날라가므로 나중에 read_csv 에서 name 옵션을 추가하여 column를 붙인다.

In [ ]:
csv_test = pd.read_csv('./SampleData/etoday.csv')
cnt = 0
m = MeCab.Tagger()

for line in csv_test["Contents"]:
    remove_email = re.compile(r'\(([^)]+\)) | [^가-힣]+@yna\.co\.kr')     # 괄호 및 안의 내용 제거 | 이메일 주소 제거
    remove_special_char = re.compile(r'[^가-힣^A-z^0-9^ ]') # 한글, 영어, 기본 문자를 제외한 문자들 제거
    text = remove_email.sub(' ', line)
    text = remove_special_char.sub(' ', text)
    
    tagged = m.parse(text)
    s = tagged.split('\n')

    result = []
    for words in s:
        # MeCab 으로 형태소 분리되면 끝이 EOS 임.
        if words == 'EOS':
            break
        word, tag = words.split(',')[0].split('\t')
        d_tag = tag.split('+')
        if (d_tag[-1] != ""):
            tag = d_tag[0]

        if tag in ["NNB", "NNBC", "VV", "VA", "VX", "VCP", "VCN", "VSV", "MAG", "MAJ", "JKS", "JKC", "JKG", 
                    "JKO", "JKB", "JKV", "JKQ", "JC", "JX", "EP", "EF", "EC", "ETN", "ETM", "XPN", "XSN", 
                    "XSV", "XSA", "SF", "SE", "SSO", "SSC", "SC", "SY", "SH", "SL", "SN", "UNA", "NA"]:
            pass
        else:
            result.append(word)
    sss = ' '.join(result)
    # 데이터 프레임으로 만들고,
    df = pd.DataFrame({
        'CNo' : csv_test['CNo'][cnt],
        'Subject' : csv_test['Subject'][cnt],
        'Contents' : [sss]
    })
    # 바로 저장
    df.to_csv("./pretreatment_data/etoday_pre.csv", encoding='utf-8',mode = 'a', index=False, header=False)
    cnt += 1
    # 얼마나 했는지 확인용
    if cnt % 1000 == 0:
        print(cnt, " 완료")